In [1]:
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from numpy import isnan

In [2]:
police_df = pd.read_csv('../data/police.csv')
police_df = police_df.drop(columns = 'Unnamed: 0')
police_df

C:\Users\jclug\AppData\Local\Temp\ipykernel_3216\3554231220.py:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  police_df = pd.read_csv('../data/police.csv')


,Date_and_Time,Date,Time,Description,Zone,RPA,Location,Latitude,longitude,Mapped_Location
0,2021-10-02T19:13:21.000,2021-10-02,19:13:21.000,BURGLARY - NON-RESIDENCE,523,9557.0,NaN,NaN,NaN,NaN
1,2021-10-03T11:25:42.000,2021-10-03,11:25:42.000,WANT OFFICER FOR INVESTIGATION / ASSISTA,513,9033.0,NaN,NaN,NaN,NaN
2,2021-10-03T00:54:02.000,2021-10-03,00:54:02.000,VEHICLE ACCIDENT - PERSONAL INJURY,323,8889.0,NaN,NaN,NaN,NaN
3,2021-10-01T08:05:43.000,2021-10-01,08:05:43.000,WANT OFFICER FOR INVESTIGATION / ASSISTA,136W,0.0,OLD HICKORY BLVD,36.079,-86.919,"{'type': 'Point', 'coordinates': [-86.919, 36...."
4,2021-09-30T18:48:20.000,2021-09-30,18:48:20.000,SUSPICIOUS PERSON,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
6943458,2017-02-11T06:17:00.000,2017-02-11,06:17:00.000,SUSPECT,415.0,4211.0,506 506,36.170,-86.779,"{'type': 'Point', 'coordinates': [-86.779, 36...."
6943459,2016-11-10T13:00:00.000,2016-11-10,13:00:00.000,SUSPECT,815.0,8051.0,I 65 N & WEDGEWOOD AVE,36.134,-86.775,"{'type': 'Point', 'coordinates': [-86.775, 36...."
6943460,2021-08-23T08:00:00.000,2021-08-23,08:00:00.000,DISPATCHED,225.0,1809.0,DICKERSON PIKE,36.230,-86.760,"{'type': 'Point', 'coordinates': [-86.76, 36.23]}"
6943461,2018-09-13T16:00:00.000,2018-09-13,16:00:00.000,NaN,513.0,9037.0,MASSMAN DR,36.150,-86.710,"{'type': 'Point', 'coordinates': [-86.71, 36.15]}"


In [3]:
zipcodes = gpd.read_file('../data/zipcodes.geojson')
zipcodes.head( )

,zip,objectid,po_name,shape_stlength,shape_starea,geometry
0,37115,1,MADISON,178783.02488886821,596553400.57885742,"MULTIPOLYGON (((-86.68725 36.31821, -86.68722 ..."
1,37216,3,NASHVILLE,75820.997821400058,188884682.28344727,"MULTIPOLYGON (((-86.73451 36.23774, -86.73425 ..."
2,37204,9,NASHVILLE,93180.292250425613,200664795.51708984,"MULTIPOLYGON (((-86.77914 36.13424, -86.77923 ..."
3,37027,11,BRENTWOOD,159760.6942933173,174978422.04101562,"MULTIPOLYGON (((-86.81258 36.06319, -86.81263 ..."
4,37064,18,FRANKLIN,28995.828320601937,46969608.005737305,"MULTIPOLYGON (((-87.02197 36.01200, -87.02140 ..."


In [6]:
print(police_df.loc[3,'Latitude'])
print(type(police_df.loc[3,'Latitude']))

36.079
<class 'numpy.float64'>


In [13]:
isnan(police_df.loc[0,'Latitude'])

True

In [19]:
geo = []

for row, value in police_df.iterrows():
    if not isnan(value.Latitude):
        coord = Point(value.longitude,value.Latitude)
        geo.append(coord)
    else:
        geo.append(None)

In [20]:
len(geo)

6943463

In [21]:
police_df['geometry'] = geo

C:\Users\jclug\anaconda3\envs\geospatial_ds6\lib\site-packages\pandas\core\dtypes\cast.py:126: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [23]:
police_df = police_df.drop(columns='Geometry')
police_df

,Date_and_Time,Date,Time,Description,Zone,RPA,Location,Latitude,longitude,Mapped_Location,geometry
0,2021-10-02T19:13:21.000,2021-10-02,19:13:21.000,BURGLARY - NON-RESIDENCE,523,9557.0,NaN,NaN,NaN,NaN,None
1,2021-10-03T11:25:42.000,2021-10-03,11:25:42.000,WANT OFFICER FOR INVESTIGATION / ASSISTA,513,9033.0,NaN,NaN,NaN,NaN,None
2,2021-10-03T00:54:02.000,2021-10-03,00:54:02.000,VEHICLE ACCIDENT - PERSONAL INJURY,323,8889.0,NaN,NaN,NaN,NaN,None
3,2021-10-01T08:05:43.000,2021-10-01,08:05:43.000,WANT OFFICER FOR INVESTIGATION / ASSISTA,136W,0.0,OLD HICKORY BLVD,36.079,-86.919,"{'type': 'Point', 'coordinates': [-86.919, 36....",POINT (-86.919 36.079)
4,2021-09-30T18:48:20.000,2021-09-30,18:48:20.000,SUSPICIOUS PERSON,NaN,NaN,NaN,NaN,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...
6943458,2017-02-11T06:17:00.000,2017-02-11,06:17:00.000,SUSPECT,415.0,4211.0,506 506,36.170,-86.779,"{'type': 'Point', 'coordinates': [-86.779, 36....",POINT (-86.779 36.17)
6943459,2016-11-10T13:00:00.000,2016-11-10,13:00:00.000,SUSPECT,815.0,8051.0,I 65 N & WEDGEWOOD AVE,36.134,-86.775,"{'type': 'Point', 'coordinates': [-86.775, 36....",POINT (-86.775 36.134)
6943460,2021-08-23T08:00:00.000,2021-08-23,08:00:00.000,DISPATCHED,225.0,1809.0,DICKERSON PIKE,36.230,-86.760,"{'type': 'Point', 'coordinates': [-86.76, 36.23]}",POINT (-86.76 36.23)
6943461,2018-09-13T16:00:00.000,2018-09-13,16:00:00.000,NaN,513.0,9037.0,MASSMAN DR,36.150,-86.710,"{'type': 'Point', 'coordinates': [-86.71, 36.15]}",POINT (-86.71 36.15)
